<a href="https://colab.research.google.com/github/Parth-349/PRODIGY_GA_01/blob/main/GA_01_Text_Genaration_with_chatgpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Step 1: Install Required Libraries
!pip install transformers datasets accelerate

# Step 2: Import Libraries
from datasets import load_dataset, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Step 3: Load Pretrained GPT-2 Tokenizer and Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 4: Upload Your Custom Dataset (one text per line)
from google.colab import files
uploaded = files.upload()

# Step 5: Load and Tokenize Dataset
# Read the content of the uploaded file
file_name = list(uploaded.keys())[0]
with open(file_name, 'r') as f:
    text_data = f.readlines()

# Create a dataset from the loaded text
dataset = Dataset.from_dict({"text": text_data})


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Step 6: Prepare Training Components
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    prediction_loss_only=True,
    fp16=True,
    report_to="none",
)

# Step 7: Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# Step 8: Save Model
model.save_pretrained("gpt2-finetuned")
tokenizer.save_pretrained("gpt2-finetuned")

# Step 9: Generate Text from the Fine-Tuned Model
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2-finetuned", tokenizer=tokenizer)
prompt = "Once upon a time"
output = generator(prompt, max_new_tokens=100, num_return_sequences=1, repetition_penalty=1.2)
print(output[0]['generated_text'])

Saving train.txt to train (6).txt


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/tmp/ipython-input-7-4289289996.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Device set to use cuda:0


Once upon a time, the world was filled with wonder.

But this one moment saw chaos and pain as all things were thrown into turmoil…
